# Libs

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import warnings
warnings.filterwarnings('ignore')

pd.set_option.display_precision = 3
pd.set_option.display_float_format = lambda x: '%.5f' % x
pd.set_option.display_max_columns = 15
pd.set_option.display_max_rows = 6

In [2]:
# def cut_azery_ntg_bal8(maps_ntg, dataset, margin_well):
#     coef = 500
#     Xmin = dataset.Xmean.min() - margin_well
#     Xmax = dataset.Xmean.max() + margin_well
#     Ymin = dataset.Ymean.min() - margin_well
#     Ymax = dataset.Ymean.max() + margin_well

#     maps_ntg = maps_ntg[(maps_ntg.X >= Xmin) &
#                         (maps_ntg.X <= Xmax) &
#                         (maps_ntg.Y >= Ymin) &
#                         (maps_ntg.Y <= Ymax)]
#     return maps_ntg

# data = pd.read_csv('avg_magnitude', header=9, delim_whitespace=True, names=['X', 'Y', 'NTGcube'])
# ntg8_azeri = cut_azery_ntg_bal8(data, df_bal8, margin_well=500)
# ntg8_azeri.to_csv('ntg8_azeri.csv', index=False)

# fig, ax = plt.subplots(figsize=(15, 10))
# ax.scatter(ntg8_azeri['X'], ntg8_azeri['Y'], c=ntg8_azeri['NTGcube'], cmap='viridis', s=1)
# # ax.scatter(ntg8_logs['Xmean'], ntg8_logs['Ymean'], s=100, c=ntg8_logs['VSH'], cmap='viridis', alpha=0.75, ec='black', lw=1)
# plt.title('NTG cube Azery')

In [1]:
# def cut_azery_ntg_bal8(maps_ntg, dataset, margin_well):
#     coef = 500
#     Xmin = dataset.Xmean.min() - margin_well
#     Xmax = dataset.Xmean.max() + margin_well
#     Ymin = dataset.Ymean.min() - margin_well
#     Ymax = dataset.Ymean.max() + margin_well

#     maps_ntg = maps_ntg[(maps_ntg.X >= Xmin) &
#                         (maps_ntg.X <= Xmax) &
#                         (maps_ntg.Y >= Ymin) &
#                         (maps_ntg.Y <= Ymax)]
#     return maps_ntg

# data_reduce = pd.read_csv('avg_magnitude_reduce', header=9, delim_whitespace=True, names=['X', 'Y', 'NTGcube'])
# ntg8_azeri_reduce = cut_azery_ntg_bal8(data_reduce, df_bal8, margin_well=500)
# ntg8_azeri_reduce.to_csv('ntg8_azeri_reduce.csv', index=False)

# fig, ax = plt.subplots(figsize=(15, 10))
# ax.scatter(ntg8_azeri_reduce['X'], ntg8_azeri_reduce['Y'], c=ntg8_azeri_reduce['NTGcube'], cmap='viridis', s=1)
# # ax.scatter(ntg8_logs['Xmean'], ntg8_logs['Ymean'], s=100, c=ntg8_logs['VSH'], cmap='viridis', alpha=0.75, ec='black', lw=1)
# plt.title('NTG cube reduce Azery')

# Bal8

In [ ]:
def cut_azery_ntg_bal8(maps_ntg, dataset, margin_well):
    coef = 500
    Xmin = dataset.Xmean.min() - margin_well
    Xmax = dataset.Xmean.max() + margin_well
    Ymin = dataset.Ymean.min() - margin_well
    Ymax = dataset.Ymean.max() + margin_well

    maps_ntg = maps_ntg[(maps_ntg.X >= Xmin) &
                        (maps_ntg.X <= Xmax) &
                        (maps_ntg.Y >= Ymin) &
                        (maps_ntg.Y <= Ymax)]
    return maps_ntg

df_bal8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v3.csv').drop(columns=['Unnamed: 0'])
df_bal8 = df_bal8[df_bal8.FORMATION != 'Balakhany VIII']
ntg8_logs = df_bal8.groupby('well').agg({'Xmean': lambda x: x.iloc[0], 
                                         'Ymean': lambda x: x.iloc[0], 
                                         'VSH':'mean'}).reset_index()

data8_reduce_tie = pd.read_csv('avg_magnitude_reduce_tie8', header=9, delim_whitespace=True, names=['X', 'Y', 'VSH_avg_cube'])
ntg8_azeri_reduce_tie = cut_azery_ntg_bal8(data8_reduce_tie, df_bal8, margin_well=500)
ntg8_azeri_reduce_tie.to_csv('ntg8_azeri_reduce_tie.csv', index=False)

fig, ax = plt.subplots(figsize=(15, 10))
ax.scatter(ntg8_azeri_reduce_tie['X'], ntg8_azeri_reduce_tie['Y'], c=ntg8_azeri_reduce_tie['VSH_avg_cube'], cmap='viridis_r', s=1)
plt.title('VSH_avg_cube reduce tie Azery');

In [ ]:
print('X step', (ntg8_azeri_reduce_tie['X'].max() - ntg8_azeri_reduce_tie['X'].min())/len(ntg8_azeri_reduce_tie['X']))
print('Y step', (ntg8_azeri_reduce_tie['Y'].max() - ntg8_azeri_reduce_tie['Y'].min())/len(ntg8_azeri_reduce_tie['Y']))

In [ ]:
def ntg_from_grcube_map(maps_var, well_coords, zone_size, var):
    df_lst = []
    for wellname in well_coords.well.unique():
        data_fm = well_coords[(well_coords.well == wellname)]
        well_x = data_fm['Xmean'].values[0].round(0)
        well_y = data_fm['Ymean'].values[0].round(0)
        map_avg = maps_var[ (maps_var.X < well_x + zone_size) &
                            (maps_var.X > well_x - zone_size) &
                            (maps_var.Y < well_y + zone_size) &
                            (maps_var.Y > well_y - zone_size)].reset_index(drop=True)
        df_avg = map_avg[var].mean()
        df = pd.DataFrame({'well': [wellname], 'Xmean': [well_x], 'Ymean': [well_y], 
                        '_'.join([var, str(zone_size)]): [df_avg]})
        df_lst.append(df)
    maps_ntg_wells_fm = pd.concat(df_lst).reset_index(drop=True)
    return maps_ntg_wells_fm
maps_ntg_wells_fm8 = ntg_from_grcube_map(ntg8_azeri_reduce_tie, ntg8_logs, 50, 'VSH_avg_cube')
map_well_join8 = maps_ntg_wells_fm8.set_index('well').join(ntg8_logs[['well','VSH']].set_index('well')).reset_index().rename(columns={'VSH':'VSHmean'})
map_well_join8.to_csv('ntg8_cube_vsh_wells.csv')

def target_selection(dataset):
    target = []
    for col in dataset.columns:
        if 'VSH_avg' in col:
            target = col
            break
    return target
target = target_selection(map_well_join8)
sns.scatterplot(data=map_well_join8, x=target, y='VSHmean')
plt.plot([0.3, 0.9], [0.3,0.9], c='red', ls='--')
plt.plot([0.3, 0.9], [0.3,0.9*1.25], c='red', ls='--')
plt.plot([0.3, 0.9], [0.3,0.9*0.75], c='red', ls='--')
plt.title('NTG cube vs VSH logs Balakhany VIII +/- 25%')
plt.grid()

# Bal10

In [ ]:
df_bal10 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_v4.csv')
df_bal10 = df_bal10[df_bal10.FORMATION != 'Balakhany X']
ntg10_logs = df_bal10.groupby('well').agg({ 'Xmean': lambda x: x.iloc[0], 
                                            'Ymean': lambda x: x.iloc[0], 
                                            'VSH':'mean'}).reset_index()

data_reduce_tie10 = pd.read_csv('avg_magnitude_reduce_tie10', header=9, delim_whitespace=True, names=['X', 'Y', 'NTGcube'])
# ntg8_azeri_reduce_tie = cut_azery_ntg_bal8(data_reduce_tie10, df_bal10, margin_well=500)
data_reduce_tie10.to_csv('ntg10_reduce_tie.csv', index=False)

fig, ax = plt.subplots(figsize=(15, 10))
ax.scatter(data_reduce_tie10['X'], data_reduce_tie10['Y'], c=data_reduce_tie10['NTGcube'], cmap='viridis', s=1)
plt.title('VSH_avg Azery')

In [ ]:
print('X step', (data_reduce_tie10['X'].max() - data_reduce_tie10['X'].min())/len(data_reduce_tie10['X']))
print('Y step', (data_reduce_tie10['Y'].max() - data_reduce_tie10['Y'].min())/len(data_reduce_tie10['Y']))

In [ ]:
def ntg_from_grcube_map(maps_var, well_coords, zone_size, var):
    df_lst = []
    for wellname in well_coords.well.unique():
        data_fm = well_coords[(well_coords.well == wellname)]
        well_x = data_fm['Xmean'].values[0].round(0)
        well_y = data_fm['Ymean'].values[0].round(0)
        map_avg = maps_var[ (maps_var.X < well_x + zone_size) &
                            (maps_var.X > well_x - zone_size) &
                            (maps_var.Y < well_y + zone_size) &
                            (maps_var.Y > well_y - zone_size)].reset_index(drop=True)
        df_avg = map_avg[var].mean()
        df = pd.DataFrame({'well': [wellname], 'Xmean': [well_x], 'Ymean': [well_y], 
                        '_'.join([var, str(zone_size)]): [df_avg]})
        df_lst.append(df)
    maps_ntg_wells_fm = pd.concat(df_lst).reset_index(drop=True)
    return maps_ntg_wells_fm
maps_ntg_wells_fm10 = ntg_from_grcube_map(data_reduce_tie10, ntg10_logs, 50, 'NTGcube')
map_well_join10 = maps_ntg_wells_fm10.set_index('well').join(ntg10_logs[['well','VSH']].set_index('well')).reset_index().rename(columns={'VSH':'VSHmean'})
map_well_join10.to_csv('ntg10_cube_vsh_wells.csv')

def target_selection(dataset):
    target = []
    for col in dataset.columns:
        if 'NTG' in col:
            target = col
            break
    return target
target = target_selection(map_well_join10)
sns.scatterplot(data=map_well_join10, x=target, y='VSHmean')
plt.plot([0.3, 0.9], [0.3,0.9], c='red', ls='--')
plt.plot([0.3, 0.9], [0.3,0.9*1.25], c='red', ls='--')
plt.plot([0.3, 0.9], [0.3,0.9*0.75], c='red', ls='--')
plt.title('NTG cube vs VSH logs Balakhany X +/- 25%')
plt.grid()

# Bal8 FU

In [88]:
data = pd.read_csv(r'C:\Petrel\exports\bal8fu\5avg_magnitude_8_10', header=9, delim_whitespace=True, names=['x', 'y', 'vsh_avg_cube'])

In [89]:
def cut_azery_ntg_bal8(maps_ntg, dataset, margin_well):
    coef = 500
    Xmin = dataset.xmean.min() - margin_well
    Xmax = dataset.xmean.max() + margin_well
    Ymin = dataset.ymean.min() - margin_well
    Ymax = dataset.ymean.max() + margin_well

    maps_ntg = maps_ntg[(maps_ntg.x >= Xmin) &
                        (maps_ntg.x <= Xmax) &
                        (maps_ntg.y >= Ymin) &
                        (maps_ntg.y <= Ymax)]
    return maps_ntg

df_bal8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v4.csv')
df_bal8.columns = df_bal8.columns.str.lower()
df_bal8 = df_bal8[df_bal8.formation != 'Balakhany VIII']
vsh8_logs = df_bal8.groupby('well').agg({'xmean': lambda x: x.iloc[0], 
                                         'ymean': lambda x: x.iloc[0], 
                                         'vsh':'mean'}).reset_index()

file_list = glob.glob(r'C:\Petrel\exports\bal8fu\*')
file_list = [file for file in file_list if '.csv' not in file]
for idx, filename in enumerate(file_list):
    suffix = file_list[idx].split('_')[-1]
    order = idx + 1
    data = pd.read_csv(filename, header=9, delim_whitespace=True, names=['x', 'y', 'vsh_avg_cube'])
    data_reduce = cut_azery_ntg_bal8(data, df_bal8, margin_well=500)
    data_reduce.to_csv(r'C:\Petrel\exports\bal8fu\\' + str(order) + 'vshmap8_' + str(suffix) + '.csv', index=False)

In [ ]:
file_list = glob.glob(r'C:\Petrel\exports\bal8fu\*.csv')
for idx, filename in enumerate(file_list):
    suffix = file_list[idx].split('_')[-1]
    order = idx + 1
    data = pd.read_csv(filename)
    fig, ax = plt.subplots(figsize=(15, 10))
    plot = ax.scatter(data['x'], data['y'], c=data['vsh_avg_cube'], cmap='viridis_r', s=1)
    plt.colorbar(plot)
    plt.title('VSH_avg_cube Bal8' + str(suffix));

In [ ]:
bal8_10top = ['B02', 'B02Z', 'B13Z', 'B14Z', 'B16Y', 'B16Z', 'B19','B27Z', 'B34', 'B34Z', 'B37', 'B38Z']

bal8_10_map = pd.read_csv(r'C:\Petrel\exports\bal8fu\\5vshmap8_10.csv')
bal8_10_map.loc[bal8_10_map['vsh_avg_cube'] > 0.5, 'color'] = 'green'
bal8_10_map.loc[(bal8_10_map['vsh_avg_cube'] > 0.25) & 
                (bal8_10_map['vsh_avg_cube'] <= 0.5), 'color'] = 'orange'
bal8_10_map.loc[bal8_10_map['vsh_avg_cube'] <= 0.25, 'color'] = 'yellow'
fig, ax = plt.subplots(figsize=(15, 10))
plot = ax.scatter(bal8_10_map['x'], bal8_10_map['y'], c=bal8_10_map['color'],s=1)
plt.scatter(vsh8_logs[vsh8_logs.well.isin(bal8_10top)]['xmean'], 
            vsh8_logs[vsh8_logs.well.isin(bal8_10top)]['ymean'], 
            s=100, c='none', alpha=1, ec='black', lw=3)
plt.scatter(vsh8_logs[~vsh8_logs.well.isin(bal8_10top)]['xmean'], 
            vsh8_logs[~vsh8_logs.well.isin(bal8_10top)]['ymean'], 
            s=50, c='none', alpha=1, ec='black', lw=1)
plt.title('vsh_avg_cube Bal8_10');